In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Udemy/ML com Python/Datasets/school_grades_weca_dataset.csv')

In [ ]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,fatherd,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [ ]:
df.shape

(649, 33)

In [ ]:
df.isnull().sum()

,0
school,0
sex,0
age,0
address,0
famsize,0
Pstatus,0
Medu,0
Fedu,0
Mjob,0
Fjob,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  fatherd     649 non-null    object
 18  activities  649 non-null    object
 19  nursery     649 non-null    object
 20  higher    

In [ ]:
df.nunique()

,0
school,2
sex,2
age,8
address,2
famsize,2
Pstatus,2
Medu,5
Fedu,5
Mjob,5
Fjob,5


In [ ]:
# Remover as variáveis categóricas e transformar em numéricas

categoricas = df.drop(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
                       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3'], axis=1)

In [ ]:
encoding_categoricas = pd.get_dummies(categoricas).astype(int)

In [ ]:
encoding_categoricas.head()

,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,fatherd_no,fatherd_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,1,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1,0
1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0
2,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,1,0
3,1,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1
4,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0


In [ ]:
encoding_categoricas.shape

(649, 43)

In [ ]:
df.drop(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian',
         'schoolsup', 'famsup', 'activities', 'nursery', 'higher', 'internet', 'romantic'], axis=1, inplace=True)

In [ ]:
df_concat = pd.concat([df, encoding_categoricas], axis=1)

In [ ]:
df_concat.head()

,age,Medu,Fedu,traveltime,studytime,failures,fatherd,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,fatherd_no,fatherd_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,no,4,3,4,1,1,3,4,0,11,11,1,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1,0
1,17,1,1,1,2,0,no,5,3,3,1,1,3,2,9,11,11,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0
2,15,1,1,1,2,0,no,4,3,2,2,3,3,6,12,13,12,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,1,0
3,15,4,2,1,3,0,no,3,2,2,1,1,5,0,14,14,14,1,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1
4,16,3,3,1,2,0,no,4,3,2,1,2,5,0,11,13,13,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0


In [ ]:
df_concat.shape

(649, 60)

In [ ]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 60 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                649 non-null    int64 
 1   Medu               649 non-null    int64 
 2   Fedu               649 non-null    int64 
 3   traveltime         649 non-null    int64 
 4   studytime          649 non-null    int64 
 5   failures           649 non-null    int64 
 6   fatherd            649 non-null    object
 7   famrel             649 non-null    int64 
 8   freetime           649 non-null    int64 
 9   goout              649 non-null    int64 
 10  Dalc               649 non-null    int64 
 11  Walc               649 non-null    int64 
 12  health             649 non-null    int64 
 13  absences           649 non-null    int64 
 14  G1                 649 non-null    int64 
 15  G2                 649 non-null    int64 
 16  G3                 649 non-null    int64 
 1

In [ ]:
df_concat.drop('fatherd', axis=1, inplace=True)

## Separação das Variáveis

In [ ]:
# Definindo as variáveis preditoras e target
y = df_concat['G3']
X = df_concat.drop('G3', axis=1)

## Aplicação do Modelo

In [ ]:
# Criação do primeiro modelo de teste
modelo1 = AdaBoostRegressor(random_state=42)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
resultado = cross_val_score(modelo1, X, y, cv=kfold, n_jobs=-1)
print(resultado.mean())

0.7857397389431636


In [24]:
# Testando outro modelo
modelo2 = BaggingRegressor(estimator=AdaBoostRegressor(random_state=42),
                           n_estimators=100,
                           n_jobs=-1,
                           random_state=42)
resultado = cross_val_score(modelo2, X, y, cv=kfold, n_jobs=-1)
print(resultado.mean())

0.8375770237891803


In [25]:
# Realizando alguns ajustes de parâmetros
modelo3 = BaggingRegressor(estimator=AdaBoostRegressor(n_estimators=300,
                                                       learning_rate=0.1,
                                                       random_state=42),
                           n_estimators=100,
                           n_jobs=-1,
                           random_state=42)
resultado = cross_val_score(modelo3, X, y, cv=kfold, n_jobs=-1)
print(resultado.mean())

0.828142269445236


In [26]:
# Teste de diferentes parâmetros
params = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.001, 0.1, 0.2, 0.3],
}

In [27]:
modelo4 = AdaBoostRegressor(random_state=42)
random_search = RandomizedSearchCV(estimator=modelo4,
                                   param_distributions=params,
                                   cv=kfold,
                                   n_jobs=-1)

random_search.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=AdaBoostRegressor(random_state=42), n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.001, 0.1, 0.2,
                                                          0.3],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]})

In [28]:
random_search.best_params_

{'n_estimators': 100, 'learning_rate': 0.01}

In [29]:
random_search.best_score_

0.8439386545785776

## Comparação desses melhores parâmetros aplicados com Bagging e sem Bagging

In [30]:
modelo5 = BaggingRegressor(estimator=AdaBoostRegressor(n_estimators=100,
                                                       learning_rate=0.01,
                                                       random_state=42),
                           n_estimators=100,
                           n_jobs=-1,
                           random_state=42)
resultado = cross_val_score(modelo5, X, y, cv=kfold, n_jobs=-1)
print(resultado.mean())

0.8456669136962658


In [31]:
modelo6 = AdaBoostRegressor(n_estimators=100,
                            learning_rate=0.01,
                            random_state=42)
resultado = cross_val_score(modelo6, X, y, cv=kfold, n_jobs=-1)
print(resultado.mean())

0.8439386545785776


## Outra Validação

In [32]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=.2, random_state=42)

modelo7 = AdaBoostRegressor(n_estimators=100,
                        learning_rate=0.01,
                        random_state=42)
modelo7.fit(X_tr, y_tr)

AdaBoostRegressor(learning_rate=0.01, n_estimators=100, random_state=42)

In [33]:
modelo8 = BaggingRegressor(estimator=AdaBoostRegressor(n_estimators=100,
                                                       learning_rate=0.01,
                                                       random_state=42),
                           n_estimators=100,
                           n_jobs=-1,
                           random_state=42)
modelo8.fit(X_tr, y_tr)

BaggingRegressor(estimator=AdaBoostRegressor(learning_rate=0.01,
                                             n_estimators=100,
                                             random_state=42),
                 n_estimators=100, n_jobs=-1, random_state=42)

In [35]:
# Avaliação do modelo 7
y_pred = modelo7.predict(X_ts)
print(f"R2: {r2_score(y_ts, y_pred)}")
print(f"MAE: {mean_absolute_error(y_ts, y_pred)}")
print(f"MSE: {mean_squared_error(y_ts, y_pred)}")

R2: 0.840284539266572
MAE: 0.7125212772931321
MSE: 1.557499810101294


In [36]:
# Avaliação do modelo 8
y_pred = modelo8.predict(X_ts)
print(f"R2: {r2_score(y_ts, y_pred)}")
print(f"MAE: {mean_absolute_error(y_ts, y_pred)}")
print(f"MSE: {mean_squared_error(y_ts, y_pred)}")

R2: 0.8500656311866853
MAE: 0.6894160135205879
MSE: 1.4621173797579592
